In [3]:
from fastapi import FastAPI
from gdeltdoc import GdeltDoc, Filters
import pandas as pd

In [4]:
app = FastAPI()

# OPCION 1:
# 3 ENDPOINTS PARA EXTRAER DATOS DE GDELT y uno para la limpieza 
@app.get("/articles/")
def get_articles(keyword: str, start_date: str, end_date: str, country: str):
    f = Filters(keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        country=[country])
    gd = GdeltDoc()
    articles = gd.article_search(f)
    df = pd.DataFrame(articles)
    df.to_csv("csv_articulos.csv") # si lo que queremos es que se nos descargue en el ordenador y obtener un mensaje de que se ha hecho bien
    return "200:ok"

@app.get("/tone/") # PARA EL TONO
def get_tone_results(keyword: str, start_date: str, end_date: str, country: str):
    f = Filters(keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        country=[country])
    gd = GdeltDoc()
    tone_results = gd.timeline_search("timelinetone", f)
    df_tono = pd.DataFrame(tone_results)
    df_tono.to_csv("csv_tono.csv")
    return "200:ok" # si lo que queremos es que se nos descargue en el ordenador y obtener un mensaje de que se ha hecho bien

@app.get("/popularity/") # PARA LA POPULARIDAD
def get_popularity_results(keyword: str, start_date: str, end_date: str, country: str):
    f = Filters( keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        country=[country])
    gd = GdeltDoc()
    popularity_results = gd.timeline_search("timelinevol", f)
    df_popularidad = pd.DataFrame(popularity_results)
    df_popularidad.to_csv("csv_popularidad.csv")
    return "200:ok" # si lo que queremos es que se nos descargue en el ordenador

# PARA LA FUNCION DE LIMPIEZA, FUNCION AUXILIAR
def limpiar_dataframe(df: pd.DataFrame):
    df_limpio= df.dropna().drop_duplicates()
    return df_limpio # creo la funcion primero, que luego podre usar para cualquier DataFrame. Metemos solo los nulos y duplicados en un primer momento

# Endpoint para limpiar los datos (este incluiría solo estas tres funciones)
@app.get("/limpieza/")
def endpoint_limpiar_dataframe(funcion_obtener_datos: str, keyword: str, start_date: str, end_date: str, country: str): # LOS PARAMETROS NECESARIOS
    if funcion_obtener_datos == "articles":
        df = get_articles(keyword, start_date, end_date, country)
    elif funcion_obtener_datos == "tone":
        df = get_tone_results(keyword, start_date, end_date, country)
    elif funcion_obtener_datos == "popularity":
        df = get_popularity_results(keyword, start_date, end_date, country)
    else:
        return {"mensaje": "Función de obtener datos no válida"}
    
    df_limpio = limpiar_dataframe(df) # Llamar a la función para limpiar el DataFrame
    
    # y lo guardamos limpio
    df_limpio.to_csv(f"csv_{funcion_obtener_datos}_limpios.csv")  
     
    return {"mensaje": f"Datos de {funcion_obtener_datos} limpiados y guardados exitosamente"}



In [5]:
# opcion dos, que LA LIMPIEZA ESTE DENTRO DE CADA LLAMADA Y RECIBAMOS LOS DATOS DIRECTAMENTE LIMPIOS, QUE ES MAS UTIL,
# PORQUE NO TENDRIA SENTIDO BAJARNOSLO SIN LIMPIAR PUDIENDO HACERLO LIMPIADOS
# pero NO TENDRIAMOS UNA FUNCION PROPIA PARA LA LIMPIEZA, CONSULTAR!!
from fastapi import FastAPI
import pandas as pd

app = FastAPI()

def limpiar_dataframe(df: pd.DataFrame): # PRIMERO LA DE LIMPIEZA, GENERAL!
    df_cleaned = df.dropna().drop_duplicates()
    return df_cleaned 

# Endpoint 1, para EXTRACCION DE LOS ARTICULOS Y LIMPIEZA
@app.get("/articles/")
def get_articles(keyword: str, start_date: str, end_date: str, country: str):
    f = Filters(keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        country=[country])
    gd = GdeltDoc()
    articles = gd.article_search(f)
    df = pd.DataFrame(articles)
    df_limpiado = limpiar_dataframe(df)   # Función para limpiar el DataFrame
    df_limpiado.to_csv("csv_articulos_limpios.csv")
    
    return {"mensaje": "Datos de artículos limpiados y guardados exitosamente"}

# Endpoint para obtener tono Y LIMPIEZA
@app.get("/tone/")
def get_tone_results(keyword: str, start_date: str, end_date: str, country: str):
    f = Filters(keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        country=[country])
    gd = GdeltDoc()
    tone_results = gd.timeline_search("timelinetone", f)
    df_tono = pd.DataFrame(tone_results)
    df_limpiado = limpiar_dataframe(df_tono)
    df_limpiado.to_csv("csv_tono_limpios.csv", index=False)
    
    return {"mensaje": "Datos de tono limpiados y guardados exitosamente"}

@app.get("/popularity/") # PARA LA EXTRACCION DE LOS DATOS DE POPULARIDAD Y LIMPIEZA 
def get_popularity_results(keyword: str, start_date: str, end_date: str, country: str):
    f = Filters( keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        country=[country])
    gd = GdeltDoc()
    popularity_results = gd.timeline_search("timelinevol", f)
    df_popularidad = pd.DataFrame(popularity_results)
    df_limpiado = limpiar_dataframe(df_popularidad)
    
    # Guardamos el DataFrame limpiado en un archivo CSV
    df_limpiado.to_csv("csv_popularidad_limpios.csv")
    
    return {"mensaje": "Datos de popularidad limpiados y guardados exitosamente"}
